In [1]:
from os import error
import sqlite3
import pandas as pd
# Create your connection.
conn = sqlite3.connect('../data/carmander.db')

df = pd.read_sql_query("SELECT * FROM kbb_listings_new", conn)
df['mileage_value'] = pd.to_numeric(df['mileage_value'].str.replace(',',''))

df_raw = df.copy()
df_raw

,accelerate,bodyStyles_0_code,bodyStyles_0_name,bodyStyleSubType_0_code,bodyStyleSubType_0_name,color_exteriorColor,color_exteriorColorSimple,description_hasMore,description_label,displacementUOM,...,owner_images_sources_28_alt,owner_images_sources_28_src,blinkerChatBean_blinkerChatName,blinkerChatBean_blinkerListingId,pricingDetail_msrp,packages_5_name,styles_3_code,packages_6_name,styles_4_code,packages_7_name
0,False,SEDAN,Sedan,COMPACT,Compact,Silver,SILVER,True,&quot;AS-IS&quot; WHOLESALE TO THE PUBLIC! Thi...,1.8,...,None,None,None,None,None,None,None,None,None,None
1,False,TRUCKS,Truck,COMPACT_CREW,Compact Crew Cab,Magnetic Gray Metallic,GRAY,True,"Clean, Well Maintained - 2018 Toyota Tacoma TR...",3.5,...,None,None,None,None,None,None,None,None,None,None
2,False,HATCH,Hatchback,COMPACT,Compact,Silver,SILVER,True,www.mycarcorner.com<br><br>Our website has vid...,1.6,...,None,None,None,None,None,None,None,None,None,None
3,False,COUPE,Coupe,COMPACT,Compact,White,WHITE,True,Selling my 2013 Hyundai Veloster Turbo. New t...,1.6,...,None,None,None,None,None,None,None,None,None,None
4,False,HATCH,Hatchback,COMPACT,Compact,Silver,SILVER,True,* WHOLE SALE UNIT !!! * ENGINE AND TRANSMISSIO...,2.0,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45310,False,COUPE,Coupe,EXOTIC,Exotic,Mythos Black Metallic,BLACK,True,Clean Carfax! Carbon Fiber Front Spoiler! Bang...,5.2,...,None,None,None,None,None,None,None,None,None,None
45311,False,COUPE,Coupe,EXOTIC,Exotic,Kemora Gray Metallic,GRAY,True,Our 2020 Audi R8 V10 performance quattro Coupe...,5.2,...,None,None,None,None,None,None,None,None,None,None
45312,False,CONVERT,Convertible,EXOTIC,Exotic,Red,RED,None,None,5.2,...,None,None,None,None,None,None,None,None,None,None
45313,False,CONVERT,Convertible,EXOTIC,Exotic,Ibis White/Black Roof,WHITE,True,Lamborghini Greenwich is pleased to offer this...,5.2,...,None,None,None,None,None,None,None,None,None,None


In [2]:
df_raw.columns
[col for col in df_raw.columns if 'trim' in col.lower()]

['trim_code', 'trim_name', 'atTrim']

In [3]:
df = df_raw.copy()

df['year'] = pd.to_numeric(df['year'], errors='coerce')
df['mileage'] = df['mileage_value'].astype(str).str.replace(',', '').fillna(-1).astype(int)
df['price'] = pd.to_numeric(df['pricingDetail_salePrice'], errors='coerce')
df = df[['vin', 'make_name', 'model_name', 'trim_name', 'year', 'mileage', 'price']]
df.columns = ['vin', 'make', 'model', 'trim', 'year', 'mileage', 'price']

display(df.dtypes)
df.groupby('make')[['year', 'price']].agg(['count', 'mean', 'std'])

vin        object
make       object
model      object
trim       object
year        int64
mileage     int64
price       int64
dtype: object

year                         price                            
         count         mean       std  count          mean           std
make                                                                    
Audi      6437  2019.828802  2.438407   6437  37073.488737  19196.601846
Honda    15149  2018.118292  2.605726  15149  20186.478183   4199.384092
Hyundai  11601  2018.929575  2.597536  11601  18011.109818   4841.043480
Toyota   12128  2018.285950  2.901027  12128  20134.165402   4217.775488

In [4]:
df = df_raw.copy()

df['year'] = pd.to_numeric(df['year'], errors='coerce')
df['mileage'] = df['mileage_value'].astype(
    str).str.replace(',', '').fillna(-1).astype(int)
df['price'] = pd.to_numeric(df['pricingDetail_salePrice'], errors='coerce')

df = df[['vin', 'make_name', 'model_name',
         'trim_name', 'year', 'mileage', 'price']]
df.columns = ['vin', 'make', 'model', 'trim', 'year', 'mileage', 'price']

df = (df
 .groupby(['make', 'model'])[['mileage', 'price']]
 .agg(['count', 'min', 'max', 'mean', 'std'])
 .reset_index())
df.columns = ['_'.join([col for col in [col1, col2] if col]) for col1, col2 in df.columns]
df = df.sort_values('mileage_count', ascending=False).reset_index(drop=True)
df = df.loc[:9]
# df = df['spec_mileage_count'].plot.hist(bins=100)
df_stat = df.copy()
df_stat

,make,model,mileage_count,mileage_min,mileage_max,mileage_mean,mileage_std,price_count,price_min,price_max,price_mean,price_std
0,Honda,Civic,4263,0,264612,51005.304246,35381.840011,4263,4500,25995,19867.957307,3997.250665
1,Toyota,Corolla,3312,0,270132,50474.805254,37048.839886,3312,3000,25995,19118.758454,3832.464016
2,Honda,CR-V,3311,0,765950,75769.876170,39459.100682,3311,7495,25995,20647.614920,4237.070635
3,Honda,Accord,2908,0,338000,69008.075653,40124.695141,2908,5000,25995,19769.426066,4419.996632
4,Toyota,Camry,2839,0,484399,62147.761184,40845.694304,2839,4754,25995,20640.940120,4253.416946
5,Hyundai,Elantra,2785,0,212988,54147.580610,37136.851255,2785,3777,25995,16255.947217,4646.824040
6,Hyundai,Tucson,2405,0,275683,51935.669854,31165.426399,2405,5995,25995,19861.925572,3974.970960
7,Hyundai,Sonata,2366,0,294613,57214.821217,40394.741596,2366,3995,25995,17504.877430,5258.659478
8,Toyota,RAV4,2361,0,260414,79354.939009,36730.155829,2361,6000,25995,21028.013977,4023.704494
9,Honda,HR-V,2204,0,171848,35536.372505,22125.371728,2204,9950,25995,22238.821688,2507.065851


In [5]:
from sqlalchemy import asc


df = df_raw.copy()
df = (df
      .query('mileage_value > 0')
      .groupby(['make_name', 'model_name', 'trim_name', 'bodyStyles_0_name', 'year'])['vin']
      .nunique()
      .reset_index()
      .rename(columns={'vin':'count'})
      .sort_values('count', ascending=False)
      .iloc[:10]
      .reset_index(drop=True))
df

,make_name,model_name,trim_name,bodyStyles_0_name,year,count
0,Toyota,Corolla,LE,Sedan,2021,755
1,Honda,Civic,LX,Sedan,2020,564
2,Toyota,Corolla,LE,Sedan,2022,459
3,Hyundai,Elantra,SEL,Sedan,2020,370
4,Toyota,Camry,SE,Sedan,2021,368
5,Toyota,Corolla,LE,Sedan,2020,314
6,Honda,Civic,Sport,Sedan,2020,303
7,Honda,CR-V,EX,Sport Utility,2020,269
8,Honda,Accord,Sport,Sedan,2020,263
9,Honda,HR-V,EX,Sport Utility,2021,261


In [6]:
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense

# df = df_raw.copy()

epochs = 32  # 1024 is ~5 minutes (444 steps)
batch_size = 32  # 1024 is ~5 minutes (444 steps)


def clean_and_preprocess(df: pd.DataFrame):
    df = df_raw.copy()
    # Assuming 'df' is a pandas DataFrame with the columns 'makeCode', 'modelCode', and 'vin'
    common_pairs = (
        df.groupby(['make_name', 'model_name'])
        ['vin']
        .nunique()
        .reset_index()
        .query('vin > 900')
        [['make_name', 'model_name']]
    )
    # If you want to filter by both 'makeCode' and 'modelCode', you need a different approach, such as:
    df = pd.merge(df, common_pairs, on=[
                  'make_name', 'model_name'], how='inner')

    df['owner_name'] = df['owner_name']
    df['owner_phone'] = df['owner_phone_value']
    df['owner_zip'] = df['owner_location_address_zip']
    df['owner_distance'] = pd.to_numeric(df['owner_distanceFromSearch'], errors='coerce')
    df['make'] = df['make_name']
    df['model'] = df['make'] + '_' + df['model_name']
    df['trim'] = df['model'] + '_' + df['trim_name']
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df['full_spec'] = df['trim'] + '_' + \
        df['year'].astype(str) + '_' + df['bodyStyles_0_name']
    df['mileage'] = df['mileage_value'].astype(
        str).str.replace(',', '').fillna(-1).astype(int)
    df['price'] = pd.to_numeric(df['pricingDetail_salePrice'], errors='coerce')

    df = df[['vin',
             'owner_name',
             'owner_phone',
             'owner_zip',
             'owner_distance',
             'make',
             'model',
             'trim',
             'year',
             'full_spec',
             'mileage',
             'price']]
    # df = df.applymap(lambda s: s.upper().replace(' ', '-') if type(s) == str else s)

    def format_strings(value):
        if isinstance(value, str):
            return value.upper().replace(' ', '-')
        else:
            return value
    df = df.map(format_strings)
    df = df[df['mileage'] > 0]

    return df


display('Cleaning data...')
df = clean_and_preprocess(df)

categorical_features = ['year', 'make', 'model', 'trim', 'full_spec']
numerical_features = ['mileage']

display('Preprocessing data...')
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])

target = df['price']
df = df.drop('price', axis=1)

display('Splitting data into training and test sets...')
X_train, X_test, y_train, y_test = train_test_split(
    df, target, test_size=0.2, random_state=42)

display('Preprocessing the datasets...')
X_train = preprocessor.fit_transform(X_train).toarray()
X_test = preprocessor.transform(X_test).toarray()

display('Defining the model...')
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

display('Compiling the model...')
model.compile(optimizer='adam', loss='mean_squared_error')

display('Training the model...')
model.fit(X_train,
          y_train,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(X_test, y_test))

display('Saving the model...')
today = datetime.now().strftime('%y%m%d-%H%M%S')
model_filename = f'../data/model_{today}_epoch{epochs}'
model.save(model_filename)

/Users/alex/code_projects/carmander_2311/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


'Cleaning data...'

'Preprocessing data...'

'Splitting data into training and test sets...'

'Preprocessing the datasets...'

...

'Training the model...'

Epoch 1/32
821/821 [==============================] - 1s 800us/step - loss: 250377760.0000 - val_loss: 17708530.0000
Epoch 2/32
821/821 [==============================] - 1s 688us/step - loss: 10881470.0000 - val_loss: 6913653.0000
Epoch 3/32
821/821 [==============================] - 1s 691us/step - loss: 5654870.0000 - val_loss: 4673570.0000
Epoch 4/32
821/821 [==============================] - 1s 697us/step - loss: 4174333.0000 - val_loss: 3860413.2500
Epoch 5/32
821/821 [==============================] - 1s 720us/step - loss: 3561599.2500 - val_loss: 3421316.5000
Epoch 6/32
821/821 [==============================] - 1s 686us/step - loss: 3210720.7500 - val_loss: 3144131.0000
Epoch 7/32
821/821 [==============================] - 1s 693us/step - loss: 2979196.7500 - val_loss: 2962040.7500
Epoch 8/32
821/821 [==============================] - 1s 775us/step - loss: 2817699.7500 - val_loss: 2841599.7500
Epoch 9/32
821/821 [==============================] - 1s 728us/step - loss: 2705853.

'Saving the model...'

INFO:tensorflow:Assets written to: ../data/model_231126-192835_epoch32/assets


INFO:tensorflow:Assets written to: ../data/model_231126-192835_epoch32/assets


In [7]:

display('Loading the model...')
model = load_model(model_filename)

# display('Selecting Toyota Prius rows from database...')
# query = 'SELECT vin, zip, phone, ownerName, year, make, model, trim, spec_mileage, pricingDetail FROM kbb_listings'
# choice_df = pd.read_sql_query(query, conn)
choice_df = df_raw.copy()

display('Cleaning and selecting data...')
choice_df = clean_and_preprocess(choice_df)

display('Preprocessing data...')
choice_X = preprocessor.transform(
    choice_df.drop(['price'], axis=1)).toarray()

display('Making predictions...')
choice_df['prediction'] = model.predict(choice_X)
choice_df['prediction'] = choice_df['prediction'].astype(int)
choice_df['mileage'] = choice_df['mileage'].astype(int)
choice_df['score'] = choice_df['prediction'] - choice_df['price']
choice_df['score'] = choice_df['score'] / 100
choice_df['score'] = choice_df['score'].astype(int)

choice_df.reset_index(inplace=True, drop=True)
# choice_df = choice_df[choice_df['price'] < 10000]
# print(choice_df.sort_values(by='score', ascending=True).tail(30))
# choice_df.to_csv('test_data.csv')

'Loading the model...'

'Cleaning and selecting data...'

'Preprocessing data...'

'Making predictions...'

1026/1026 [==============================] - 0s 419us/step


In [8]:
choice_df.columns

Index(['vin', 'owner_name', 'owner_phone', 'owner_zip', 'owner_distance',
       'make', 'model', 'trim', 'year', 'full_spec', 'mileage', 'price',
       'prediction', 'score'],
      dtype='object')

In [13]:
df = choice_df.copy()
df = (df
      [df['make'] == 'AUDI']
      [df['year'] > 2015]
      [df['mileage'] < 75000]
      [df['owner_distance'] < 50]
      .sort_values('price')
      .reset_index(drop=True))
df.head(30)

/var/folders/gj/0nqbxjvd6tv9zp5dm3_wd5c80000gn/T/ipykernel_83237/4179636328.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = (df
/var/folders/gj/0nqbxjvd6tv9zp5dm3_wd5c80000gn/T/ipykernel_83237/4179636328.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = (df
/var/folders/gj/0nqbxjvd6tv9zp5dm3_wd5c80000gn/T/ipykernel_83237/4179636328.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = (df


,vin,owner_name,owner_phone,owner_zip,owner_distance,make,model,trim,year,full_spec,mileage,price,prediction,score
0,WA1ANAFYXJ2114838,ROMEO-AUTO-OUTLET,5187606753,12804,40.80,AUDI,AUDI_Q5,AUDI_Q5_2.0T-PREMIUM,2018,AUDI_Q5_2.0T-PREMIUM_2018_SPORT-UTILITY,74032,20500,22081,15
1,WA1BNAFY2J2060908,BEDARD-BROS.-HONDA,4137769559,01225,28.16,AUDI,AUDI_Q5,AUDI_Q5_2.0T-PREMIUM-PLUS,2018,AUDI_Q5_2.0T-PREMIUM-PLUS_2018_SPORT-UTILITY,70889,23995,23236,-7
2,WA1BNAFY3J2120100,MCGOVERN-VOLVO-CARS-OF-ALBANY,8382176649,12205,9.52,AUDI,AUDI_Q5,AUDI_Q5_2.0T-PREMIUM-PLUS,2018,AUDI_Q5_2.0T-PREMIUM-PLUS_2018_SPORT-UTILITY,50803,26250,24372,-18
3,WA1ANAFY4L2096369,AUDI-ALBANY,5182506024,12110,4.91,AUDI,AUDI_Q5,AUDI_Q5_2.0T-PREMIUM,2020,AUDI_Q5_2.0T-PREMIUM_2020_SPORT-UTILITY,44946,28999,28917,0
4,WA1BNAFYXL2008459,KEELER-PRE-OWNED,5189529491,12110,7.91,AUDI,AUDI_Q5,AUDI_Q5_2.0T-PREMIUM-PLUS,2020,AUDI_Q5_2.0T-PREMIUM-PLUS_2020_SPORT-UTILITY,48702,29995,30442,4
5,WA1BAAFY9M2134633,SUBARU-OF-TROY,8332328717,12180,2.04,AUDI,AUDI_Q5,AUDI_Q5_2.0T-PREMIUM-PLUS,2021,AUDI_Q5_2.0T-PREMIUM-PLUS_2021_SPORT-UTILITY,40259,30868,32346,14
6,WA1ENAFY7L2090227,AUDI-ALBANY,5182506024,12110,4.91,AUDI,AUDI_Q5,AUDI_Q5_2.0T-TITANIUM-PREMIUM-PLUS,2020,AUDI_Q5_2.0T-TITANIUM-PREMIUM-PLUS_2020_SPORT-...,36392,33300,32989,-3
7,WA1BAAFY3M2012317,AUDI-ALBANY,5182506024,12110,4.91,AUDI,AUDI_Q5,AUDI_Q5_2.0T-PREMIUM-PLUS,2021,AUDI_Q5_2.0T-PREMIUM-PLUS_2021_SPORT-UTILITY,38478,33500,32447,-10
8,WA1AAAFYXM2017512,AUDI-ALBANY,5182506024,12110,4.91,AUDI,AUDI_Q5,AUDI_Q5_2.0T-PREMIUM,2021,AUDI_Q5_2.0T-PREMIUM_2021_SPORT-UTILITY,31243,35000,32407,-25
9,WA1AAAFY3M2133196,GENESIS-OF-ALBANY,5182247201,12205,9.52,AUDI,AUDI_Q5,AUDI_Q5_2.0T-PREMIUM,2021,AUDI_Q5_2.0T-PREMIUM_2021_SPORT-UTILITY,21408,35997,32963,-30
